In [3]:
import matplotlib.pyplot as plt 
import numpy as np
import random
import keras
%matplotlib inline

Using TensorFlow backend.


In [4]:
# Load training text file
Data_train = np.loadtxt('training_data.txt', skiprows = 1)
X_train = Data_train[:, 1:]
y_train = Data_train[:, 0]
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (20000, 1000)
y_train shape: (20000,)


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras import regularizers

# Build sequential model
model = Sequential()

# Hidden layers
model.add(Dense(500, activation="tanh", input_shape=(1000,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation="tanh"))

# Output layer
model.add(Dense(1, activation="sigmoid"))
# Print a summary
model.summary()
# Compile
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 500)               500500    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 101       
Total params: 550,701
Trainable params: 550,701
Non-trainable params: 0
_________________________________________________________________


In [21]:
fit = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2, shuffle=True, verbose=1)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 4s 257us/step - loss: 0.4032 - acc: 0.8187 - val_loss: 0.3485 - val_acc: 0.8490
Epoch 2/10
16000/16000 [==============================] - 4s 231us/step - loss: 0.3286 - acc: 0.8563 - val_loss: 0.3519 - val_acc: 0.8450
Epoch 3/10
16000/16000 [==============================] - 4s 228us/step - loss: 0.3078 - acc: 0.8663 - val_loss: 0.3525 - val_acc: 0.8472
Epoch 4/10
16000/16000 [==============================] - 4s 234us/step - loss: 0.2893 - acc: 0.8735 - val_loss: 0.3598 - val_acc: 0.8475
Epoch 5/10
16000/16000 [==============================] - 4s 229us/step - loss: 0.2694 - acc: 0.8815 - val_loss: 0.3586 - val_acc: 0.8490
Epoch 6/10
16000/16000 [==============================] - 4s 225us/step - loss: 0.2463 - acc: 0.8959 - val_loss: 0.3636 - val_acc: 0.8482
Epoch 7/10
16000/16000 [==============================] - 4s 241us/step - loss: 0.2232 - acc: 0.9069 - val_loss: 0.3728 - v

In [7]:
## Printing the accuracy of our model, according to the loss function specified in model.compile above
score = model.evaluate(X_train, y_train, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

20000/20000 [==============================] - 1s 65us/step
Test score: 0.000114548230696
Test accuracy: 0.99995
